## Priority Queue

Creates a priority queue of time series to be analysed based on their probability of being an astrophysical source.

#### Import packages and read in pre-processed dataframes/arrays

In [46]:
import sklearn.ensemble
import sklearn.multiclass
import sklearn.model_selection
import numpy as np
import pandas as pd
import os

unlabelledDf = pd.read_pickle('./processed.files/raw/unlabelledDfnew')
preProcFeats = np.load('./processed.files/labelled/featuresnew.npy')
preProcLabels = np.load('./processed.files/labelled/labelsnew.npy')
rawfeats = np.load('./processed.files/raw/featuresnew.npy')

#### Use the RandomForest method to fit the OneVsRest classifier on all of the data

In [47]:
randForest = sklearn.multiclass.OneVsRestClassifier(sklearn.ensemble.RandomForestClassifier()) #classifier for multiple labels

clf = sklearn.multiclass.OneVsRestClassifier(estimator=sklearn.ensemble.RandomForestClassifier(bootstrap=True, 
            class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-1,
            oob_score=False, random_state=420, verbose=0,
            warm_start=False),
            n_jobs=1)

fitmodel = clf.fit(preProcFeats, preProcLabels)

#print np.argwhere(np.isfinite(rawfeats) != True)

In [48]:
#print unlabelledDf.columns.values

#### Predict probabilities to be in class 9, add them to the dataframe, sort by descending probability, export dataframe and write to text file

In [49]:
pred = fitmodel.predict_proba(rawfeats)
pred9 = pred[:,7]

print pred9.shape
print unlabelledDf.shape

unlabelledDf['P(class9)'] = pred9

sortedDf = unlabelledDf.sort_values('P(class9)', axis = 0,  ascending = False)

filterbanks = sortedDf[['filterbank', 'datfile', 'BinFactor', 'bestDM', 'Label', 'P(class9)', 'Buffer']]
filterbanks = filterbanks.reset_index()
print filterbanks.columns

filterbanks.to_pickle('/home/inigo/pulseClassifier/scripts/priorityqueue')

base_filename = 'priorityqueue.txt'
with open(os.path.join('/home/inigo/pulseClassifier/scripts/', base_filename),'w') as outfile:
    filterbanks.to_string(outfile)

(102078,)
(102078, 492)
Index([u'index', u'filterbank', u'datfile', u'BinFactor', u'bestDM', u'Label',
       u'P(class9)', u'Buffer'],
      dtype='object')
